In [5]:
import numpy as np
import heapq
import pandas as pd
import math

# use a heap to store the best sequences found
class SequenceNode:
    def __init__(self, fitness, sequence):
        self.fitness = fitness
        self.sequence = sequence

    def __lt__(self, other):
        return self.fitness < other.fitness

def random_solution(vertices: int) -> list:
    """
    Generates a random solution for the TSP problem.
    """
    solution = np.random.permutation(vertices)
    # return to first node
    return np.append(solution, solution[0])

def get_tour_length(tour: list, graph):
    """
    Fitness function for TSP solution
    """
    cost = 0
    for i in range(len(tour)):
            if i < len(tour) - 1:
                cost += graph[tour[i]][tour[i + 1]]
    return cost

def get_best_sequence_in_solution(solution, graph, sequence_length):
    """
    Returns the best sequence in the solution based on the fitness function.
    """
    best_sequence = []
    best_fitness = np.inf
    for i in range(len(solution)):
        if i < len(solution) - sequence_length:
            sequence = solution[i:i+sequence_length]
            fitness = get_tour_length(sequence, graph)
            if fitness < best_fitness:
                best_sequence = sequence
                best_fitness = fitness
    return best_sequence, best_fitness

def avoid_mutation(mutation_sequences, best_sequences):
    """
    Avoid mutating the best sequences found in the previous generations.
    """
    for sequence in best_sequences:
        for mutation_sequence in mutation_sequences:
            if np.array_equal(sequence.sequence, mutation_sequence) or np.array_equal(sequence.sequence, mutation_sequence[::-1]):
                return True
    return False


def add_best_sequence(solution, best_sequences_heap):
    """
    Adds one of the random best sequences to the solution.
    """
    best_sequences = heapq.nsmallest(10, best_sequences_heap)
    
    # choose a random sequence from the best sequences
    random_sequence = np.random.choice(best_sequences)
    for i in range(len(solution) - 1):
        # identify the index to swap
        if solution[i] == random_sequence.sequence[0]:
            swap_index = i + 1
            swap_element = random_sequence.sequence[1]
            # swap the edges
            modified_solution = swap_elements(solution, swap_index, swap_element)
            modified_solution[-1] = modified_solution[0]
            return modified_solution

# update to use index() or np.where to get the index for the target element
def swap_elements(solution, index: int, element: int):
    """
    This function will swap the element at index with the element in the solution.
    """
    target_index = np.where(solution == element)[0][0]
    solution[index], solution[target_index] = solution[target_index], solution[index]
    return solution

def guided_best_mutation(solution: list, best_sequences_heap, mutation_rate=0.1):
    """
    This function will mutate the solution based on the best sequences found in the previous generations.
    Mutation will be avoided on the top 10 best sequences.
    Will consider passing a parameter to control how many best sequences to consider.
    """
    
    n = len(solution)

    best_sequences = heapq.nsmallest(10, best_sequences_heap)

    for i in range(2, n - 1):
        if np.random.rand() < mutation_rate:
            j = np.random.randint(n)
            # check if the solution is in the best sequences
            # need to check the sequences around index j and i
            mutation_sequences = [solution[i:(i+2)], solution[(i-1):(i+1)], solution[j:(j+2)], solution[(j-1):(j+1)]]
            print(mutation_sequences)
            if avoid_mutation(mutation_sequences, best_sequences):
                continue
            solution[i], solution[j] = solution[j], solution[i]

    solution[-1] = solution[0]
    # after random mutations, the solution may not be feasible. We need to fix any duplicates
    # solution = feasible_solution(solution)
    return solution
 

def euclidean_distance(point_1, point_2):
    x_distance = (point_1['x'] - point_2['x'])
    y_distance = (point_1['y'] - point_2['y'])
    return math.sqrt(x_distance**2 + y_distance**2)

def get_adjacency_matrix(data: pd.DataFrame, vertices: int):
    matrix = [[0] * vertices for _ in range(vertices)]

    for i in range(vertices):
        for j in range(i + 1, vertices):
            distance = euclidean_distance(data.iloc[i], data.iloc[j])
            # print(euclidean(data.iloc[i], data.iloc[j]))
            matrix[i][i] = 0
            matrix[i][j] = distance
            matrix[j][i] = distance
    
    return matrix

In [8]:
data = pd.read_csv('tsp/medium.csv', names=['x', 'y'])
graph = get_adjacency_matrix(data, 10)

# example of how to use the heap in the algorithm
best_sequences = []
best_sequence_set = set()

# add a sequence to the heap
solution = random_solution(10)
print(solution)
best_sequence, best_fitness = get_best_sequence_in_solution(solution, graph, 2)

node = SequenceNode(best_fitness, best_sequence)
if tuple(best_sequence) not in best_sequence_set:
    heapq.heappush(best_sequences, node)
    # add the sequence in forward and reverse order to the set
    best_sequence_set.add(tuple(best_sequence))
    best_sequence_set.add(tuple(best_sequence[::-1]))

for sequence in heapq.nsmallest(10, best_sequences):
    print(sequence.sequence)
# heapq.heappop(best_sequences).sequence

[8 1 3 5 7 2 9 0 4 6 8]
[9 0]
{(9, 0), (0, 9)}


In [7]:
solution_2 = random_solution(10)
print(solution_2)

modified_solution_2 = add_best_sequence(solution_2, best_sequences)
print(modified_solution_2)


[4 1 8 7 9 5 2 0 6 3 4]
[4 1 8 5 9 7 2 0 6 3 4]
